In [17]:
import os
from collections import defaultdict
import numpy as np

import gym
import gym_maze



# EXPERIMENT CONFIGURATION

In [18]:
MAZES = ["F1", "F2", "F3", "T2", "T3", "4", "5", "7", "Z", "XYZ"] 
# MAZES = ["F1"]

In [24]:
MAZE_PATH = 0
MAZE_OBSTACLE = 1
MAZE_REWARD = 9

full_metrics = []

for m in MAZES:
    maze = f'Maze{m}-v0'

    env = gym.make(maze)
    matrix = env.matrix

    # size
    size = matrix.size

    # density
    flat_matrix = matrix.reshape(-1)
    obstacle_number = np.count_nonzero(flat_matrix == MAZE_OBSTACLE)
    path_number = np.count_nonzero(flat_matrix == MAZE_PATH)
    density = obstacle_number / size

    # distance to food
    X = matrix.shape[1]
    Y = matrix.shape[0]

    def get_reward_pos():
        for i in range(Y):
            for j in range(X):
                if(matrix[i, j] == MAZE_REWARD):
                    return(i, j)

    PATHS = []
    for i in range(Y):
        for j in range(X):
            if(matrix[i, j] == MAZE_PATH):
                PATHS.append((i, j))



    def get_possible_neighbour_cords(pos_y, pos_x):
        n = ((pos_y - 1, pos_x), 4)
        ne = ((pos_y - 1, pos_x + 1), 5)
        e = ((pos_y, pos_x + 1), 6)
        se = ((pos_y + 1, pos_x + 1), 7)
        s = ((pos_y + 1, pos_x), 0)
        sw = ((pos_y + 1, pos_x - 1), 1)
        w = ((pos_y, pos_x - 1), 2)
        nw = ((pos_y - 1, pos_x - 1), 3)

        return [n, ne, e, se, s, sw, w, nw]

    def get_next_cords(pos_y, pos_x, action):
        if(action == 4): return (pos_y + 1, pos_x)
        if(action == 5): return (pos_y + 1, pos_x - 1)
        if(action == 6): return (pos_y, pos_x - 1)
        if(action == 7): return (pos_y - 1, pos_x - 1)
        if(action == 0): return (pos_y - 1, pos_x)
        if(action == 1): return (pos_y - 1, pos_x + 1)
        if(action == 2): return (pos_y, pos_x + 1)
        if(action == 3): return (pos_y + 1, pos_x + 1)

        raise ValueError

    optimal_actions = []

    root_node = get_reward_pos()

    def is_included(cords, level):
        return any(op_cords[0] == cords[0] and op_cords[1] == cords[1] and level != op_level for op_cords, _, op_level in optimal_actions)


    def get_optimal_actions_to(node, level):
        neighbour_cords = get_possible_neighbour_cords(node[0], node[1])

        next_level_cords = []
        for (pos_y, pos_x), action in neighbour_cords:
            if (not is_included((pos_y, pos_x), level)) and matrix[pos_y, pos_x] == MAZE_PATH:
                optimal_actions.append(((pos_y, pos_x), action, level))
                next_level_cords.append((pos_y, pos_x))

        return next_level_cords

    LEVEL = 0
    next_level_cords = get_optimal_actions_to(root_node, LEVEL)

    while len(next_level_cords) > 0:
        LEVEL += 1
        new_next_level_cords = []
        for nlc in next_level_cords:
            new_next_level_cords += get_optimal_actions_to(nlc, LEVEL)

        next_level_cords = new_next_level_cords

    positions_actions = defaultdict(list)
    for cords, a, _ in optimal_actions: positions_actions[cords].append(a)
    
    distances = []
    for path in PATHS:
        distance = 0
        cords = path

        while True:
            distance += 1
            action = positions_actions[cords][0]
            next_cords = get_next_cords(cords[0], cords[1], action)
            
            next_value = matrix[next_cords[0], next_cords[1]]

            if(next_value == MAZE_OBSTACLE):
                raise ValueError
            
            if(next_value == MAZE_REWARD):
                break

            cords = next_cords

        distances.append(distance)


    longest_distance = max(distances)
    avg_distance = sum(distances) / len(distances)
    metrics = [size, obstacle_number, path_number, density, longest_distance, avg_distance]
    full_metrics.append(metrics)
    print(f"{maze} metrics: {metrics}")

np.savetxt("maze_stats.csv", full_metrics, delimiter=",", fmt='%.3f')
    

MazeF1-v0 metrics: [24, 18, 5, 0.75, 3, 1.8]
MazeF2-v0 metrics: [30, 23, 6, 0.7666666666666667, 4, 2.5]
MazeF3-v0 metrics: [36, 27, 8, 0.75, 5, 3.375]
MazeT2-v0 metrics: [42, 34, 7, 0.8095238095238095, 4, 2.7142857142857144]
MazeT3-v0 metrics: [54, 44, 9, 0.8148148148148148, 6, 3.6666666666666665]
Maze4-v0 metrics: [64, 37, 26, 0.578125, 5, 3.5]
Maze5-v0 metrics: [81, 44, 36, 0.5432098765432098, 8, 4.611111111111111]
Maze7-v0 metrics: [81, 45, 35, 0.5555555555555556, 10, 6.542857142857143]
MazeZ-v0 metrics: [196, 112, 83, 0.5714285714285714, 14, 7.120481927710843]
MazeXYZ-v0 metrics: [225, 135, 89, 0.6, 18, 7.831460674157303]


## METRICS